In [1]:
import os
import gzip
import json
import pprint
from collections import defaultdict
from tqdm import tqdm

In [2]:
print(os.listdir("./download_train"))
train_files = [os.path.join("./download_train", file) for file in os.listdir("./download_train")] 
train_files

['SearchQA.jsonl.gz', 'NewsQA.jsonl.gz', 'TriviaQA.jsonl.gz', 'NaturalQuestions.jsonl.gz', 'SQuAD.jsonl.gz', 'HotpotQA.jsonl']


['./download_train/SearchQA.jsonl.gz',
 './download_train/NewsQA.jsonl.gz',
 './download_train/TriviaQA.jsonl.gz',
 './download_train/NaturalQuestions.jsonl.gz',
 './download_train/SQuAD.jsonl.gz',
 './download_train/HotpotQA.jsonl']

In [3]:
print(os.listdir("./download_in_domain_dev"))
in_domain_dev_files = [os.path.join("./download_in_domain_dev", file) for file in os.listdir("./download_in_domain_dev")] 
in_domain_dev_files

['SearchQA.jsonl.gz', 'NewsQA.jsonl.gz', 'TriviaQA.jsonl.gz', 'NaturalQuestions.jsonl.gz', 'HotpotQA.jsonl.gz', 'SQuAD.jsonl.gz']


['./download_in_domain_dev/SearchQA.jsonl.gz',
 './download_in_domain_dev/NewsQA.jsonl.gz',
 './download_in_domain_dev/TriviaQA.jsonl.gz',
 './download_in_domain_dev/NaturalQuestions.jsonl.gz',
 './download_in_domain_dev/HotpotQA.jsonl.gz',
 './download_in_domain_dev/SQuAD.jsonl.gz']

In [4]:
print(os.listdir("./download_out_of_domain_dev"))
out_domain_dev_files = [os.path.join("./download_out_of_domain_dev", file) for file in os.listdir("./download_out_of_domain_dev")] 
out_domain_dev_files

['RelationExtraction.jsonl', 'RACE.jsonl.gz', 'BioASQ.jsonl.gz', 'TextbookQA.jsonl', 'DuoRC.jsonl.gz', 'DROP.jsonl.gz']


['./download_out_of_domain_dev/RelationExtraction.jsonl',
 './download_out_of_domain_dev/RACE.jsonl.gz',
 './download_out_of_domain_dev/BioASQ.jsonl.gz',
 './download_out_of_domain_dev/TextbookQA.jsonl',
 './download_out_of_domain_dev/DuoRC.jsonl.gz',
 './download_out_of_domain_dev/DROP.jsonl.gz']

In [5]:
from bert import tokenization
tokenizer = tokenization.FullTokenizer(
     "./vocab.txt"
    ,do_lower_case = False
)

In [6]:
def open_file(filename) :
    if filename.split('.')[-1] == "gz" :
        file = gzip.open(filename,'rt')
    else :
        file = open(filename, 'rt')
    return file

def parse_data(filename) :
    
    num_lines = sum(1 for line in open_file(filename))
    data = open_file(filename)
    context_length = []
    query_length = []
    answer_length = []

    token_index = defaultdict(lambda: 0)
    unk_set = dict()

    for i, line in tqdm(enumerate(data), total = num_lines) :

        if i == 0 :
            continue
        
        try :
            jsondata = json.loads(line)

            context, unk_list = tokenizer.tokenize(jsondata['context'])

            for token in context :
                token_index[token] += 1
            
            for token, sub_tokens in unk_list :
                if token not in unk_set :
                    unk_set[token] = sub_tokens
            
            context_length.append(len(context))

            for _, q in enumerate(jsondata['qas']) :

                query, unk_list = tokenizer.tokenize(q['question'])

                for token in query :
                    token_index[token] += 1
                query_length.append(len(query))
                
                for token, sub_tokens in unk_list :
                    if token not in unk_set :
                        unk_set[token] = sub_tokens

                answer, _ = tokenizer.tokenize(q['answers'][0])
                answer_length.append(len(answer))

        except :
            continue

    info = {}
    info["Num. context"] = len(context_length)
    info["Num. query"] = len(query_length)
    info["Num. tokens"] = len(token_index)
    info["Avg. context length"] = sum(context_length) / len(context_length)
    info["Avg. query length"] = sum(query_length) / len(query_length)
    info["Avg. answer length"] = sum(answer_length) / len(answer_length)
    
    return token_index, info, unk_set

In [7]:
data = {}
for file in (train_files + out_domain_dev_files) :
    print(file)
    token_index, info, unk_set = parse_data(file)
    
    token_index_dict = dict()
    for k, v in token_index.items() :
        token_index_dict[k] = v
    
    data[file] = [token_index_dict, info, unk_set]

./download_train/SearchQA.jsonl.gz


100%|██████████| 117385/117385 [25:59<00:00, 75.26it/s] 


./download_train/NewsQA.jsonl.gz


100%|██████████| 11429/11429 [02:08<00:00, 88.77it/s] 


./download_train/TriviaQA.jsonl.gz


...

  4%|▍         | 9/203 [00:00<00:02, 86.44it/s]

./download_out_of_domain_dev/DuoRC.jsonl.gz


  7%|▋         | 21/282 [00:00<00:01, 205.42it/s]

./download_out_of_domain_dev/DROP.jsonl.gz


100%|██████████| 282/282 [00:01<00:00, 202.18it/s]


In [12]:
import pickle
with open("./data.pickle", 'wb') as handle:
    pickle.dump(data, handle)

In [10]:
for file in (train_files + out_domain_dev_files) :
    print(file)
    pprint.pprint(data[file][1])
    try :
        unknum = data[file][0]['[UNK]']
    except :
        unknum = 0
    print("Num unique [UNK] :", len(data[file][2]))
    print("frequency  [UNK] :", unknum)
    pprint.pprint(data[file][2])
    print('')

./download_train/SearchQA.jsonl.gz
{'Avg. answer length': 2.7910277380222177,
 'Avg. context length': 1082.8945256593743,
 'Avg. query length': 19.140717644653446,
 'Num. context': 117384,
 'Num. query': 117384,
 'Num. tokens': 26902}
Num unique [UNK] : 1
frequency  [UNK] : 1
{'⅔': []}

./download_train/NewsQA.jsonl.gz
{'Avg. answer length': 5.858926645091693,
 'Avg. context length': 673.2030101505076,
 'Avg. query length': 8.378087918015103,
 'Num. context': 11428,
 'Num. query': 74160,
 'Num. tokens': 26372}
Num unique [UNK] : 44
frequency  [UNK] : 61
{'AgÃ¼ero': ['A', '##g'],
 'ArchipiÃ©lago': ['Arch', '##ip', '##i'],
 'BrontÃ': ['B', '##ron', '##t'],
 'CafÃ©': ['C', '##af'],
 'ChloÃ©': ['Ch', '##lo'],
 'DesafÃo': ['Des', '##af'],
 'DreÃ¯b': ['Dr', '##e'],
 'EnrÃquez': ['En', '##r'],
 'EspaÃ±ol': ['E', '##sp', '##a'],
 'FranÃ': ['Fr', '##an'],
 'FrÃ©dÃ©ric': ['Fr'],
 'GuantÃ': ['G', '##uant'],
 'HermÃ¨s': ['Her', '##m'],
 'HespÃ©rides': ['He', '##sp'],
 'IrÃ¨ne': ['I', '##r'],
 'Jos

 'Joseｿｽfs': ['Jose'],
 'JosÃ¨phe': ['Jo', '##s'],
 'Jörg': ['Jo'],
 'Jumping™': ['Jump', '##ing'],
 'Jürgen': ['Ju'],
 'JÃ³n': ['J'],
 'JÃÂ¼rgen': ['J'],
 'KENNATΩ': ['K', '##EN', '##NA', '##T'],
 'KRAGH\uf02a': ['K', '##RA', '##G', '##H'],
 'Kaplanoǧlu': ['Kaplan', '##o'],
 'Kaŝka': ['Ka'],
 'Kenobi™': ['Ken', '##ob', '##i'],
 'Khởi': ['K', '##h'],
 'Kiefer▲Kathy': ['Ki', '##ef', '##er'],
 'KingdomпїЅ': ['Kingdom', '##п', '##ї'],
 'Kingｿｽfs': ['King'],
 'Kitemark™': ['Kit', '##ema', '##rk'],
 'KitzbÃ¼hel': ['Kit', '##z', '##b'],
 'Kong™': ['Kong'],
 'KrisztiÃ': ['Kris', '##z', '##ti'],
 'Kr̥ṣṇa': ['K', '##r'],
 'KÃ': ['K'],
 'LatÃ©ral': ['La', '##t'],
 'Learning™': ['Learning'],
 'Lemminkäinen': ['Le', '##mm', '##ink', '##a'],
 'LendMe™': ['Len', '##d', '##M', '##e'],
 'Lettingweb™': ['Let', '##ting', '##we', '##b'],
 'Leveneâ€™s': ['Lev', '##ene', '##â', '##€'],
 'Library\u0871': ['Library'],
 'LibÃ©rÃ©â': ['Li', '##b'],
 'Life™': ['Life'],
 'Location↓': ['Location'],
 'London\uf

 'nĭkō': ['n'],
 'nĭm': ['n'],
 'nĭn': ['n'],
 'nĭng': ['n'],
 'nĭnˈəvə': ['n'],
 'nĭs': ['n'],
 'nĭz': ['n'],
 'nŭdyĕˈzhdə': ['n', '##ŭ', '##dy'],
 'nɐsiˈmẽtu': ['n', '##ɐ', '##si', '##ˈ', '##m'],
 'nɒʃ': ['n'],
 'nəˈmɒnɨk': ['n', '##ə', '##ˈ', '##m'],
 'nəˈstɜr': ['n', '##ə', '##ˈ', '##st'],
 'nəˈstɜːʃəm': ['n', '##ə', '##ˈ', '##st'],
 'nəˈθɒl': ['n', '##ə', '##ˈ', '##θ'],
 'nəˌfaɪdz': ['n', '##ə'],
 'nəˌvɔr': ['n', '##ə'],
 'nəˌvɪk': ['n', '##ə'],
 'nɜrˈvoʊ': ['n'],
 'nɪˌiː': ['n', '##ɪ'],
 'nʕm': ['n'],
 'nˌbɜr': ['n'],
 'n̪ˠə': ['n'],
 'nṯ̣r': ['n'],
 'n™sir': ['n'],
 'of┬áthe': ['of'],
 'omniprÃ¤sent': ['o', '##m', '##ni', '##p', '##r'],
 'oneâ€™s': ['one', '##â', '##€'],
 'operaｿｽfs': ['opera'],
 'operaｿｽh': ['opera'],
 'or̄kh': ['or'],
 'otherｿｽfs': ['other'],
 'out΄': ['out'],
 'oōzˌdā': ['o', '##ō', '##z'],
 'oʊˈnaɪroʊˌmænsi': ['o', '##ʊ', '##ˈ', '##na', '##ɪ', '##ro', '##ʊ'],
 'oˈneiroˌmancer': ['o', '##ˈ', '##ne', '##iro'],
 'o̞u': ['o'],
 'o͝olēvā': ['o'],
 'o͞o': ['o'],
 

 'ˈɒʃ': ['ˈ'],
 'ˈɕɛˌlanˀ': ['ˈ', '##ɕ', '##ɛ'],
 'ˈɛmˌpɛg': ['ˈ', '##ɛ', '##m'],
 'ˈɛpǝlɛt': ['ˈ', '##ɛ', '##p'],
 'ˈɛpɪˌgræm': ['ˈ', '##ɛ', '##p', '##ɪ'],
 'ˈɛpɪˌɡræm': ['ˈ', '##ɛ', '##p', '##ɪ'],
 'ˈɛθənɒl': ['ˈ', '##ɛ', '##θ', '##ə', '##n'],
 'ˈɜr': ['ˈ'],
 'ˈɟurtʃaːɲ': ['ˈ'],
 'ˈɡnɒkɪ': ['ˈ', '##ɡ', '##n'],
 'ˈɡɑrˌɡɔɪl': ['ˈ', '##ɡ', '##ɑ', '##r'],
 'ˈɡɑɹ': ['ˈ', '##ɡ', '##ɑ'],
 'ˈɡɒdənˌɔːf': ['ˈ', '##ɡ'],
 'ˈɡɛtᵻsbɜːrɡ': ['ˈ', '##ɡ', '##ɛ', '##t'],
 'ˈɡʏnt': ['ˈ', '##ɡ'],
 'ˈɦɛɾmɑn̪': ['ˈ'],
 'ˈʃrɒpʃər': ['ˈ', '##ʃ', '##r'],
 'ˈʃrɒpʃɪər': ['ˈ', '##ʃ', '##r'],
 'ˈʃɒːhe': ['ˈ', '##ʃ'],
 'ˈʃɔrtˌstɒp': ['ˈ', '##ʃ', '##ɔ', '##rt'],
 'ˈʃɜrlɒk': ['ˈ', '##ʃ'],
 'ˈʃӕnti': ['ˈ', '##ʃ'],
 'ˈʕomɑɾˤ': ['ˈ'],
 'ˈʝanis': ['ˈ'],
 'ˈθrɛdˌnid': ['ˈ', '##θ', '##r', '##ɛ', '##d'],
 'ˈθərzˌdā': ['ˈ', '##θ', '##ə', '##rz'],
 'ˈӕmplifai': ['ˈ'],
 'ˌ': [],
 'ˌArgoˈnautic': [],
 'ˌEspeˈrantist': [],
 'ˌalchemˈistic': [],
 'ˌamplifiˈcation': [],
 'ˌapoˈgean': [],
 'ˌautobioˈgraphic': [],
 'ˌautobiˈographe

 'ريالاً': ['ر', '##ي', '##ا', '##ل', '##ا'],
 'ريښكۍ': ['ر', '##ي'],
 'رَحْمَه': ['ر', '##َ', '##ح'],
 'رَزَقْنَاكُمْ': ['ر', '##َ', '##ز', '##َ', '##ق'],
 'رَقْصَةُ': ['ر', '##َ', '##ق'],
 'رَّحِيمٌ': ['ر', '##َ'],
 'رِثائِيّه': ['ر', '##ِ', '##ث', '##ا', '##ئ', '##ِ', '##ي'],
 'رِحْلَةٌ': ['ر', '##ِ', '##ح'],
 'زَلْزالي': ['ز', '##َ', '##ل'],
 'زړه': ['ز'],
 'سؤالك': ['س'],
 'سابقاً': ['س', '##ا', '##ب', '##ق', '##ا'],
 'سكّة': ['س', '##ك'],
 'سندھ': ['س', '##ن', '##د'],
 'سنڌو': ['س', '##ن'],
 'سنکھیا': ['س', '##ن', '##ک'],
 'سنګر': ['س', '##ن'],
 'سيلۍ': ['س', '##ي', '##ل'],
 'سَفْرَة': ['س', '##َ', '##ف'],
 'سَفْرَةٌ': ['س', '##َ', '##ف'],
 'سَمْت': ['س', '##َ', '##م'],
 'سُحبت': ['س'],
 'سُر': ['س'],
 'سُم': ['س'],
 'سِتِرْيُو': ['س', '##ِ', '##ت', '##ِ', '##ر'],
 'سِيْرَةٌ': ['س', '##ِ', '##ي'],
 'سټير': ['س'],
 'سړى': ['س'],
 'شac뢱': ['ش', '##ac'],
 'شوراوزوږ': ['ش', '##و', '##ر', '##ا', '##و', '##ز', '##و'],
 'شومړې': ['ش', '##و', '##م'],
 'شَاموْ': ['ش', '##َ', '##ا', '##م',

...

 '반': [],
 '방정환': [],
 '방탄': [],
 '버뮤다': [],
 '벨기에': [],
 '보보': [],
 '보았다': [],
 '보이': [],
 '부처님': [],
 '북미': [],
 '비무장': [],
 '사랑': ['사'],
 '사월': ['사'],
 '사정': ['사'],
 '삼성': [],
 '샘': [],
 '생': [],
 '샤인': [],
 '서울': [],
 '석가': [],
 '선생님': [],
 '설': [],
 '설날': [],
 '션': [],
 '소년': [],
 '소년단': [],
 '소리': [],
 '수': [],
 '스타일': [],
 '승': [],
 '시대': [],
 '신부': [],
 '신세계': [],
 '신정': [],
 '아가씨': [],
 '아니야': [],
 '아사달': [],
 '아저씨': [],
 '아주머니': [],
 '아줌마': [],
 '악마': [],
 '암행': [],
 '어린이': [],
 '어사': [],
 '언니': [],
 '에': [],
 '여러분': [],
 '연인': [],
 '열정': [],
 '예멘': [],
 '오빠': [],
 '오신': [],
 '올림픽': [],
 '왜': [],
 '우리말': [],
 '월드컵': [],
 '은': [],
 '이니': ['이'],
 '이라': ['이'],
 '이루마': ['이'],
 '이번': ['이'],
 '이봉수': ['이'],
 '이승만': ['이'],
 '이인수': ['이'],
 '인간': [],
 '인민군': [],
 '일본': [],
 '자모': [],
 '자음': [],
 '작성': [],
 '잠든': [],
 '장마': [],
 '재통일': [],
 '전자': [],
 '전쟁': [],
 '절': [],
 '제': [],
 '제헌절': [],
 '조국': [],
 '조선': [],
 '조선말': [],
 '조선어': [],
 '주인': [],
 '지대': [],
 '지부티': [],
 '채종기': [],
 '처

 '詞': [],
 '試': [],
 '話': [],
 '語': [],
 '説': [],
 '議': [],
 '讀': [],
 '让': [],
 '试': [],
 '话': [],
 '语': [],
 '豆': [],
 '豊': [],
 '豔': [],
 '起': [],
 '越': [],
 '趙': [],
 '軍': [],
 '通': [],
 '逸': [],
 '過': [],
 '邑': [],
 '那': [],
 '都': [],
 '鄴': [],
 '釁': [],
 '重': [],
 '針': [],
 '銖': [],
 '鋪': [],
 '铺': [],
 '长': [],
 '閏': [],
 '閩': [],
 '關': [],
 '门': [],
 '问': [],
 '闽': [],
 '陋': [],
 '院': [],
 '陳': [],
 '陵': [],
 '隶': [],
 '隸': [],
 '集': [],
 '雍': [],
 '雙': [],
 '雨': [],
 '雲': [],
 '震': [],
 '韓': [],
 '韩': [],
 '音': [],
 '頓': [],
 '題': [],
 '顯': [],
 '顿': [],
 '题': [],
 '风': [],
 '館': [],
 '騷': [],
 '驗': [],
 '验': [],
 '體': [],
 '鬱': [],
 '魂': [],
 '魄': [],
 '魏': [],
 '鮮': [],
 '鱻': [],
 '鲁': [],
 '鲜': [],
 '齉': [],
 '龘': [],
 '龢': [],
 'ꀕ': [],
 '고려파': [],
 '고신': [],
 '그리스도': [],
 '기독교도': [],
 '기장': [],
 '나라이름': [],
 '대': [],
 '대한예수교장로회': [],
 '물': [],
 '부': [],
 '사람': ['사'],
 '소': [],
 '수': [],
 '슈퍼': [],
 '아래': [],
 '아비': [],
 '인': [],
 '작을': [],
 '조국해방전쟁': [],
 '조선족': [],
 '컴보이

 'ਫੌਜਾ': [],
 'ਬੁਘਚੂ': [],
 'ਭੰਗੜਾ': [],
 'ਮਲਕੀਤ': [],
 'ਮਾਣਕ': [],
 'ਮਾਨ': [],
 'ਮੋਨਿਕਾ': [],
 'ਰੱਬਾਬ': [],
 'ਸਿੰਘ': [],
 'ਸੇਨੀ': [],
 'આનંદ': [],
 'ગરબા': [],
 'ગુજરાતી': [],
 'ઢોલ': [],
 'તાહિર': [],
 'નવરાત્રી': [],
 'પાઠક': [],
 'ફાલ્ગુની': [],
 'માં': [],
 'શાહ': [],
 'શિલા': [],
 'ଗିନି': [],
 'ପଣା': [],
 'ବେଲ': [],
 'ହେମଙ୍ଗଦ': [],
 'அனு': [],
 'அவென்யூ': [],
 'ஆதிபராசக்தி': [],
 'ஆனந்த்': [],
 'இனிய': [],
 'இயந்திரா': [],
 'இராஜ': [],
 'இலங்கை': [],
 'உயிராக': [],
 'உயிரோடு': [],
 'உழவன்': [],
 'எக்ஸ்ப்ரெஸ்': [],
 'என்': [],
 'எம்': [],
 'கந்தையா': ['க'],
 'கமலேஸ்வரன்': ['க', '##ம', '##ல'],
 'கரகாட்டம்': ['க', '##ர', '##க', '##ா'],
 'கருன்': ['க', '##ர', '##ு'],
 'காதலன்': ['க', '##ா', '##த', '##ல'],
 'கிராஞ்சி': ['க', '##ி', '##ர', '##ா'],
 'கிரிஷ்': ['க', '##ி', '##ர', '##ி'],
 'கிருஷ்ணதாசி': ['க', '##ி', '##ர', '##ு'],
 'குழி': ['க', '##ு'],
 'கோட்டை': ['க'],
 'கோபாலகிருஷ்ணன்': ['க'],
 'கோயில்': ['க'],
 'சக்கரை': [],
 'சக்திவனேஸ்வரர்': [],
 'சங்கர்': [],
 'சங்கீத': [],
 'சந்த

 '復': [],
 '微': [],
 '徳': [],
 '德': [],
 '志': [],
 '忠': [],
 '念': [],
 '怒': [],
 '思': [],
 '急': [],
 '性': [],
 '怪': [],
 '恋': [],
 '恩': [],
 '恵': [],
 '悟': [],
 '悠': [],
 '悪': [],
 '悲': [],
 '情': [],
 '惟': [],
 '惠': [],
 '惡': [],
 '意': [],
 '愚': [],
 '愽': [],
 '慄': [],
 '慎': [],
 '慜': [],
 '慧': [],
 '慶': [],
 '憲': [],
 '憶': [],
 '應': [],
 '戀': [],
 '戊': [],
 '戏': [],
 '成': [],
 '我': [],
 '战': [],
 '戦': [],
 '戯': [],
 '戰': [],
 '所': [],
 '扉': [],
 '手': [],
 '扮': [],
 '承': [],
 '技': [],
 '折': [],
 '报': [],
 '拉': [],
 '拓': [],
 '拳': [],
 '持': [],
 '振': [],
 '捕': [],
 '授': [],
 '掌': [],
 '掛': [],
 '探': [],
 '携': [],
 '撃': [],
 '撒': [],
 '撮': [],
 '擦': [],
 '攻': [],
 '放': [],
 '政': [],
 '故': [],
 '敎': [],
 '敏': [],
 '教': [],
 '敦': [],
 '敬': [],
 '敷': [],
 '斎': [],
 '斗': [],
 '料': [],
 '斯': [],
 '旅': [],
 '族': [],
 '早': [],
 '旭': [],
 '时': [],
 '昂': [],
 '昆': [],
 '昇': [],
 '昊': [],
 '易': [],
 '映': [],
 '春': [],
 '昭': [],
 '是': [],
 '時': [],
 '晃': [],
 '晋': [],
 '晓': [],
 '景': [],
 '晶': [],


 '임수정': [],
 '임윤아': [],
 '잇': [],
 '자': [],
 '자유선언': [],
 '잔혹사': [],
 '장우영': [],
 '장진호': [],
 '전': [],
 '전투': [],
 '정글의': [],
 '정말': [],
 '정윤호': [],
 '정재하야': [],
 '정재헌': [],
 '정준영밴드': [],
 '정진영': [],
 '정찬성': [],
 '제네시스': [],
 '제이제이': [],
 '젝스키스': [],
 '조국해방전쟁': [],
 '조선4': [],
 '조수미': [],
 '조양호': [],
 '좋아': [],
 '좋은': [],
 '주식회사': [],
 '중소기업청': [],
 '중소벤처기업부': [],
 '쥬블스': [],
 '지대': [],
 '지연': [],
 '짝사랑': [],
 '차학연': [],
 '청해진해운': [],
 '최강연승': [],
 '최영의': [],
 '최재봉': [],
 '최준홍': [],
 '침몰': [],
 '카라': [],
 '컴보이': [],
 '케이팝': [],
 '코리아': [],
 '퀴즈쇼': [],
 '크레이지': [],
 '킹': [],
 '토요일': [],
 '투포케이': [],
 '투피엠': [],
 '투하트': [],
 '트러블메이커': [],
 '트로트': [],
 '파수꾼': [],
 '파이터': [],
 '파파이스': [],
 '펀치': [],
 '펌프': [],
 '평양': [],
 '포옹': [],
 '표류': [],
 '프로젝트': [],
 '프리스틴': [],
 '플란다스의': [],
 '하녀': [],
 '하이트맥주': [],
 '하이포': [],
 '학교': [],
 '학살사건': [],
 '한국': ['한'],
 '한국전쟁': ['한'],
 '한문': ['한'],
 '한반도': ['한'],
 '한아름': ['한'],
 '한자': ['한'],
 '해장국': [],
 '해피': [],
 '현대': [],
 '현대백화점': [],
 '현대자동차': [],


In [13]:
# show unique token of datasets
token_set_dict = dict()
for file in (train_files + out_domain_dev_files) :
    print(file)
    
    common_token_set = set()
    for file2 in (train_files + out_domain_dev_files) :
        if file == file2 : continue
        common_token_set = common_token_set.union(set(list(data[file2][0].keys())))
    
    intersection = set(list(data[file][0].keys())) - common_token_set
    print(list(intersection))
    print(len(intersection), end = "\n\n")

./download_train/SearchQA.jsonl.gz
['Walkover', 'ochreous', 'scooted']
3

./download_train/NewsQA.jsonl.gz
[]
0

./download_train/TriviaQA.jsonl.gz
['♦', '##რ', 'ן', 'ს', 'ş', '∩', 'ि', '##ছ', '##∞', 'ो', 'ã', '##ἐ', 'ი', '##ể', '⁹', '##ḳ', '⇒', 'ὰ', 'ỹ', '##ŷ', '##⁵', '##↑', '##⁶', '##ƒ', '●', '##ო', 'ί', '│', '##µ', '##●', '##ʳ', 'ℓ', '∘', '##য়', 'Ľ', 'ğ', 'ा', '##│', '∧', '##ე', 'щ', 'º', '##■', '↔', 'ạ', '##♦', 'ώ', '##І', '##È', '##आ', '##√', '##Е', 'ष', '##ℓ', '͡', 'ئ', '『', 'յ', '##♥', '।', '郡', '##ი', '##そ', '##₅', '##ἀ', '■', '─', '##Ф', 'ţ', '##₃', '³', 'ǫ', '##⁹', 'े', '##¬', '##Μ', 'ო', '亻', 'ु', 'ë', '##₄', '##₆', '##─', '##⁻', '##མ', 'ა', '##₁', 'რ', '##ლ', '##÷', 'ḳ', '##⁸', 'ी', '##ს', '₂', '##≡', '』', '##ⁱ', '##∂', 'ė', '##Γ', 'ŵ', '##☆']
103

./download_train/NaturalQuestions.jsonl.gz
['⊂', '##ẩ', 'ལ', '̃', '##À', 'れ', '##ག', '이', '##Ψ', 'ũ', '사', '☉', '́', 'ய', 'ḍ', 'Kaladin', '##Ḥ', 'ッ', '♣', '##ལ', '♠', '##ི', 'ς', '##せ', 'ე', '##Κ', 'ர', 'ར', '##이', 'ư', '॥', 'ー',